In [1]:
%matplotlib inline  
import glob
import zipfile
#import multiprocessing as mp
import gc
from collections import defaultdict
import csv
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from itertools import combinations_with_replacement, product
from tqdm import tqdm_notebook, tqdm

datadir = "/home/jhyun/workbox/dataset/00_PATSTAT_2019SP/"


In [2]:
_base_dir = "/home/jhyun/workbox/00_scopus_data_test/output/20190803/country_pair/"
outfile_collab_pat_applt = _base_dir + "no_collab_pat_applt_20190909.tsv"
outfile_total_pat_applt = _base_dir + "no_total_pat_applt_20190909.tsv"
outfile_collab_pair_applt = _base_dir + "no_collab_pair_applt_20190909.tsv"
outfile_citation_pair_applt = _base_dir + "no_citation_pair_applt_20190909.tsv"

outfile_collab_pat_invt = _base_dir + "no_collab_pat_invt_20190909.tsv"
outfile_total_pat_invt = _base_dir + "no_total_pat_invt_20190909.tsv"
outfile_collab_pair_invt = _base_dir + "no_collab_pair_invt_20190909.tsv"
outfile_citation_pair_invt = _base_dir + "no_citation_pair_invt_20190909.tsv"

In [3]:
tls201_files = glob.glob(datadir + "tls201*.zip")
# 
print(tls201_files)
tls201 = pd.concat((pd.read_csv(f, compression="zip", usecols=['appln_id', 'appln_kind', 'appln_filing_year', 'earliest_pat_publn_id']) for f in tls201_files))

['/home/jhyun/workbox/dataset/00_PATSTAT_2019SP/tls201_part02.zip', '/home/jhyun/workbox/dataset/00_PATSTAT_2019SP/tls201_part01.zip']


In [4]:
tls201[0:10]

,appln_id,appln_kind,appln_filing_year,earliest_pat_publn_id
0,51068767,A,1921,404508984
1,51068768,A,1954,327196099
2,51068769,A,1965,315093345
3,51068770,A,1974,313734403
4,51068771,A,1983,309140092
5,51068772,A,1990,303942018
6,51068773,A,2003,291502691
7,51068774,A,1921,325705881
8,51068775,A,1942,327777845
9,51068776,A,1954,327008208


In [5]:
tls227_files = glob.glob(datadir + "tls227*.zip") # Person ID for the publications
tls227 = pd.concat((pd.read_csv(f, compression="zip",  usecols=['person_id', 'pat_publn_id','applt_seq_nr','invt_seq_nr']) for f in tls227_files))

In [6]:
tls227[0:10]

,person_id,pat_publn_id,applt_seq_nr,invt_seq_nr
0,1,6460,1,0
1,1,43605,1,0
2,1,51584,1,0
3,1,51596,1,0
4,1,51639,1,0
5,1,51656,1,0
6,1,60103,1,0
7,1,60235,1,0
8,1,84415,1,0
9,1,84801,2,0


In [7]:
tls201_tls227 = tls201.merge(tls227, left_on="earliest_pat_publn_id", right_on = "pat_publn_id")

In [8]:
tls201_tls227[0:10]

,appln_id,appln_kind,appln_filing_year,earliest_pat_publn_id,person_id,pat_publn_id,applt_seq_nr,invt_seq_nr
0,51068767,A,1921,404508984,39081207,404508984,0,1
1,51068767,A,1921,404508984,46260860,404508984,1,0
2,51068768,A,1954,327196099,39081208,327196099,0,1
3,51068768,A,1954,327196099,46260861,327196099,1,0
4,51068769,A,1965,315093345,17095760,315093345,1,0
5,51068769,A,1965,315093345,37196334,315093345,0,1
6,51068769,A,1965,315093345,37196335,315093345,0,2
7,51068770,A,1974,313734403,6013659,313734403,1,0
8,51068770,A,1974,313734403,8260939,313734403,0,1
9,51068770,A,1974,313734403,8260940,313734403,0,2


In [9]:
tls201_tls227.groupby("appln_kind").count()

,appln_id,appln_filing_year,earliest_pat_publn_id,person_id,pat_publn_id,applt_seq_nr,invt_seq_nr
appln_kind,,,,,,,
A,193877396,193877396,193877396,193877396,193877396,193877396,193877396
B,509,509,509,509,509,509,509
C,5608,5608,5608,5608,5608,5608,5608
D,756664,756664,756664,756664,756664,756664,756664
E,370,370,370,370,370,370,370
F,1324003,1324003,1324003,1324003,1324003,1324003,1324003
H,12,12,12,12,12,12,12
K,6854,6854,6854,6854,6854,6854,6854
L,618,618,618,618,618,618,618


In [10]:
tls201_tls227 = (tls201_tls227[:][(tls201_tls227["appln_kind"] == "A ") & (tls201_tls227["appln_filing_year"] >= 2000) 
                                      & (tls201_tls227["appln_filing_year"] <= 2018)])

In [11]:
del(tls201)
del(tls227)

In [12]:
tls201_tls227[0:10]

,appln_id,appln_kind,appln_filing_year,earliest_pat_publn_id,person_id,pat_publn_id,applt_seq_nr,invt_seq_nr
14,51068773,A,2003,291502691,33875011,291502691,0,1
15,51068773,A,2003,291502691,38212768,291502691,0,2
16,51068773,A,2003,291502691,50994869,291502691,1,0
36,51068780,A,2003,297330700,18303081,297330700,0,2
37,51068780,A,2003,297330700,30356769,297330700,2,0
38,51068780,A,2003,297330700,30388270,297330700,3,0
39,51068780,A,2003,297330700,30394003,297330700,1,0
40,51068780,A,2003,297330700,44412461,297330700,0,3
41,51068780,A,2003,297330700,50949635,297330700,0,1
42,51068781,A,2006,278524233,5211119,278524233,1,0


In [13]:
tls206_files = glob.glob(datadir + "tls206*.zip")
tls206 = pd.concat((pd.read_csv(f, compression="zip",  usecols=['person_id', 'person_ctry_code']) for f in tls206_files))

In [14]:
tls206[0:10]

,person_id,person_ctry_code
0,1,FI
1,2,FI
2,3,FI
3,4,FI
4,5,FI
5,6,FI
6,7,GB
7,8,GB
8,9,GB
9,10,GB


In [15]:
tls201_tls227_tls206 = tls201_tls227.merge(tls206, on="person_id")

In [16]:
tls201_tls227_tls206[0:10]

,appln_id,appln_kind,appln_filing_year,earliest_pat_publn_id,person_id,pat_publn_id,applt_seq_nr,invt_seq_nr,person_ctry_code
0,51068773,A,2003,291502691,33875011,291502691,0,1,AU
1,42106056,A,2004,284726561,33875011,284726561,0,2,AU
2,47822747,A,2002,285731622,33875011,285731622,0,1,AU
3,48403058,A,2002,291502689,33875011,291502689,0,1,AU
4,51068773,A,2003,291502691,38212768,291502691,0,2,AU
5,48403058,A,2002,291502689,38212768,291502689,0,2,AU
6,51068773,A,2003,291502691,50994869,291502691,1,0,US
7,51068780,A,2003,297330700,18303081,297330700,0,2,JP
8,52144240,A,2003,286091565,18303081,286091565,0,2,JP
9,53241579,A,2001,297330697,18303081,297330697,0,2,JP


In [17]:
len(tls201_tls227_tls206)

114838626

In [18]:
#del(tls206)
#del(tls201_tls227)

In [19]:
gc.collect()

25

In [20]:
applt_ctry_dict = tls201_tls227_tls206[tls201_tls227_tls206["applt_seq_nr"] != 0].groupby("pat_publn_id")["person_ctry_code"].apply(set).to_dict()

In [21]:
invt_ctry_dict = tls201_tls227_tls206[tls201_tls227_tls206["invt_seq_nr"] != 0].groupby("pat_publn_id")["person_ctry_code"].apply(set).to_dict()

In [22]:
no_collab_pat_applt = defaultdict(int)
no_total_pat_applt = defaultdict(int)
no_collab_pair_applt = defaultdict(int)

for k, v in tqdm_notebook(applt_ctry_dict.items()):
    countries = set([str(x).strip() for x in v if str(x).strip() != ""])
    for cou in countries:
        no_total_pat_applt[cou] += 1
        if(len(countries) > 1):
            no_collab_pat_applt[cou] += 1
    pairs = list(combinations_with_replacement(countries, 2))
    for cou_pair in pairs:
        cou_pair = tuple(sorted(cou_pair))
        no_collab_pair_applt[cou_pair] += 1 

In [23]:
no_collab_pat_invt = defaultdict(int)
no_total_pat_invt = defaultdict(int)
no_collab_pair_invt = defaultdict(int)

for k, v in tqdm_notebook(invt_ctry_dict.items()):
    countries = set([str(x).strip() for x in v if str(x).strip() != ""])
    for cou in countries:
        no_total_pat_invt[cou] += 1
        if(len(countries) > 1):
            no_collab_pat_invt[cou] += 1
    pairs = list(combinations_with_replacement(countries, 2))
    for cou_pair in pairs:
        cou_pair = tuple(sorted(cou_pair))
        no_collab_pair_invt[cou_pair] += 1 

In [25]:
f_out = open(outfile_collab_pat_applt, "w")
for k, v in no_collab_pat_applt.items():
    f_out.write(k + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

f_out = open(outfile_collab_pair_applt, "w")
for k, v in no_collab_pair_applt.items():
    f_out.write("\t".join(k) + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

f_out = open(outfile_collab_pat_invt, "w")
for k, v in no_collab_pat_invt.items():
    f_out.write(k + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

f_out = open(outfile_collab_pair_invt, "w")
for k, v in no_collab_pair_invt.items():
    f_out.write("\t".join(k) + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

In [ ]:
tls212_files = glob.glob(datadir + "tls212*.zip") # citations
tls212 = pd.concat((pd.read_csv(f, compression="zip",  usecols=['pat_publn_id', 'cited_pat_publn_id', 'pat_citn_seq_nr'], quoting=csv.QUOTE_NONE) for f in tls212_files))

In [ ]:
tls212_filtered = tls212[(tls212["cited_pat_publn_id"] != 0) & (tls212["pat_citn_seq_nr"] != 0)]
del(tls212)

In [ ]:
no_citation_pair_applt = defaultdict(int)
no_citation_pair_invt = defaultdict(int)

for row in tqdm_notebook(tls212_filtered.itertuples(), total = len(tls212_filtered)):
    citedby = getattr(row, "pat_publn_id")
    cited = getattr(row, "cited_pat_publn_id")
    
    if(citedby in applt_ctry_dict and cited in applt_ctry_dict):
        countries_1 = set([str(x).strip() for x in applt_ctry_dict[citedby] if str(x).strip() != ""])
        countries_2 = set([str(x).strip() for x in applt_ctry_dict[cited] if str(x).strip() != ""])
        pairs = list(product(*([countries_1, countries_2])))
        for cit_cou_pair in pairs:
            no_citation_pair_applt[cit_cou_pair] += 1    
    
    if(citedby in invt_ctry_dict and cited in invt_ctry_dict):
        countries_1 = set([str(x).strip() for x in invt_ctry_dict[citedby] if str(x).strip() != ""])
        countries_2 = set([str(x).strip() for x in invt_ctry_dict[cited] if str(x).strip() != ""])
        pairs = list(product(*([countries_1, countries_2])))
        for cit_cou_pair in pairs:
            no_citation_pair_invt[cit_cou_pair] += 1    

In [ ]:
f_out = open(outfile_total_pat_applt, "w")
for k, v in no_total_pat_applt.items():
    f_out.write(k + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

f_out = open(outfile_citation_pair_applt, "w")
for k, v in no_citation_pair_applt.items():
    f_out.write("\t".join(k) + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

f_out = open(outfile_total_pat_invt, "w")
for k, v in no_total_pat_invt.items():
    f_out.write(k + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()

f_out = open(outfile_citation_pair_invt, "w")
for k, v in no_citation_pair_invt.items():
    f_out.write("\t".join(k) + "\t" + str(v) + "\n")
    #print(k, v)
f_out.close()